<a href="https://colab.research.google.com/github/evertonmj/howtocode_bigdata/blob/main/kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kafka-python

In [2]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
kafka_version='kafka_2.12-3.8.0'
topic_name = 'reco-train'
kafka_bootstrap_servers = 'localhost:9092'
kafka = kafka_version + '/bin'

In [4]:

!curl -sSOL https://downloads.apache.org/kafka/3.8.0/{kafka_version}.tgz
!tar -xzf {kafka_version}.tgz
!ls

'Career Mode female player datasets - FIFA 16-22.xlsx'
'Career Mode player datasets - FIFA 15-22.xlsx'
 female_players_16.csv
 female_players_17.csv
 female_players_18.csv
 female_players_19.csv
 female_players_20.csv
 female_players_21.csv
 female_players_22.csv
 fifa-22-complete-player-dataset.zip
 kafka_2.12-3.8.0
 kafka_2.12-3.8.0.tgz
 players_15.csv
 players_16.csv
 players_17.csv
 players_18.csv
 players_19.csv
 players_20.csv
 players_21.csv
 players_22.csv
 sample_data
 spark-3.5.2-bin-hadoop3
 spark-3.5.2-bin-hadoop3.tgz
 spark-3.5.2-bin-hadoop3.tgz.1
 spark-3.5.2-bin-hadoop3.tgz.2
 spark-3.5.2-bin-hadoop3.tgz.3
 spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar
 spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar.1
 spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar.2
 spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar.3
 spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar


In [5]:
!ls {kafka_version}
!./{kafka_version}/bin/zookeeper-server-start.sh -daemon ./{kafka_version}/config/zookeeper.properties
!./{kafka_version}/bin/kafka-server-start.sh -daemon ./{kafka_version}/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

bin  config  libs  LICENSE  licenses  logs  NOTICE  site-docs
Waiting for 10 secs until kafka and zookeeper services are up and running


In [6]:
!ps -ef | grep kafka

root      109373  108245  0 02:05 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root      109375  109373  0 02:05 ?        00:00:00 grep kafka


In [7]:
!./{kafka_version}/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic reco-train
!./{kafka_version}/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic reco-test


Error while executing topic command : Topic 'reco-train' already exists.
[2024-09-05 02:05:11,237] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'reco-train' already exists.
 (org.apache.kafka.tools.TopicCommand)
Error while executing topic command : Topic 'reco-test' already exists.
[2024-09-05 02:05:14,697] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'reco-test' already exists.
 (org.apache.kafka.tools.TopicCommand)


In [ ]:
!./{kafka_version}/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic reco-train
!./{kafka_version}/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic reco-test

In [ ]:
# !pip install kagle
!kaggle datasets download -d stefanoleone992/fifa-22-complete-player-dataset
# !wget -O ml_ratings.csv https://github.com/sparsh-ai/reco-data/blob/master/MovieLens_100K_ratings.csv?raw=true
!unzip fifa-22-complete-player-dataset.zip
!ls

In [ ]:
players_df_15 = pd.read_csv('players_15.csv')
# players_df_16 = pd.read_csv('players_16.csv')
# players_df_17 = pd.read_csv('players_17.csv')
# players_df_18 = pd.read_csv('players_18.csv')
# players_df_19 = pd.read_csv('players_19.csv')
# players_df_20 = pd.read_csv('players_20.csv')
# players_df_21 = pd.read_csv('players_21.csv')
# players_df_22 = pd.read_csv('players_22.csv')
# players_df = pd.concat([players_df_15, players_df_16, players_df_17, players_df_18, players_df_19, players_df_20, players_df_21, players_df_22])
players_df = players_df_15
players_df.head()

In [ ]:
len(players_df)

In [ ]:
players_df.columns.values.tolist()

In [ ]:
!pip install kafka-python

import json
from kafka import KafkaProducer

def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                         key_serializer=lambda k: json.dumps(k).encode(),
                         value_serializer=lambda v: json.dumps(v).encode()) # Changed the key serializer to convert the tuple key to bytes
  for message, key in items:
    producer.send(topic_name, key=key, value=message).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
# records = players_df.to_json(orient='records', lines=True)
# players_df = players_df.set_index(columns=players_df.columns['sofifa_id'])
# players_df.set_index('sofifa_id', inplace=True)
# records = players_df.set_index('sofifa_id')[players_df.columns.values.toList()].apply(tuple, axis=1).to_dict()
# records = players_df.to_dict('index')
players_df = players_df[~players_df.index.duplicated(keep='first')]

players_df = players_df.reset_index().set_index(['sofifa_id', 'short_name'])
records = players_df.to_dict('index')


# records = list(players_df.itertuples(index=True, name=None))
# print(records[:10])
# records = json.loads(records)
# records
write_to_kafka("reco-train", records.items())

In [ ]:
!pip install confluent-kafka

In [ ]:
from confluent_kafka import Consumer, TopicPartition
from concurrent.futures import ThreadPoolExecutor

consumer = Consumer({"bootstrap.servers": "localhost:9092", "group.id": "reco-train"})

def get_partition_size(topic_name: str, partition_key: int):
    topic_partition = TopicPartition(topic_name, partition_key)
    low_offset, high_offset = consumer.get_watermark_offsets(topic_partition)
    partition_size = high_offset - low_offset
    return partition_size

def get_topic_size(topic_name: str):
    topic = consumer.list_topics(topic=topic_name)
    partitions = topic.topics[topic_name].partitions
    workers, max_workers = [], len(partitions) or 1

    with ThreadPoolExecutor(max_workers=max_workers) as e:
        for partition_key in list(topic.topics[topic_name].partitions.keys()):
            job = e.submit(get_partition_size, topic_name, partition_key)
            workers.append(job)

    topic_size = sum([w.result() for w in workers])
    return topic_size

print(get_topic_size('reco-train'))

In [ ]:
# !./{kafka_version}/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic recoTraing --from-beginning --max-messages 100
!./{kafka_version}/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic reco-train --group=reco-train --from-beginning --max-messages 1000

In [ ]:
print(get_topic_size('reco-train'))

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar -xvf spark-3.5.2-bin-hadoop3.tgz
!pip install findspark
!pip install pyspark

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10-assembly_2.13/3.5.2/spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar"

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-10-assembly_2.13-3.5.2.jar pyspark-shell'

In [ ]:
!pip install pyspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
# from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time

kafka_topic_name = "reco-train"
kafka_bootstrap_servers = 'localhost:9092'

In [ ]:
!pip install --force-reinstall pyspark==2.4.6


In [ ]:
# Create a new SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.2")\
    .getOrCreate()

# Now you can import and use KafkaUtils
from pyspark.streaming.kafka import KafkaUtils

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,5)

kafka_topic_name = "reco-train"
kafka_bootstrap_servers = 'localhost:9092'

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

lines = kvs.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(' '))
counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
counts.pprint()
ssc.start()
# stream will run for 50 sec
ssc.awaitTerminationOrTimeout(50)
ssc.stop()
sc.stop()